# Tutorial 4: solving optimal power flow with MadNLP

With the previous tutorial, we have now all the elements to implement
the optimal power flow problem with MadNLP. It remains to add to the model
the following elements:

- an objective penalizing the cost to run each power generator.
- line flow constraints;
- voltage angle constraints;

We start the tutorial again by importing the usual packages:

In [1]:
using LinearAlgebra
using SparseArrays

using NLPModels
using ExaModels

using JLD2

include("utils.jl")

get_block_reordering (generic function with 1 method)

We import a small instance:

In [2]:
DATA_DIR = "/home/fpacaud/dev/examodels-tutorials/instances"
data = JLD2.load(joinpath(DATA_DIR, "case9.jld2"))["data"]
ngen = length(data.gen)
nbus = length(data.bus)
nlines = length(data.branch)

9

## Optimal power flow model

We implement the AC optimal power flow problem using ExaModels in the
following function `acopf_model`:

In [3]:
function acopf_model(
    data;
    backend = nothing,
    T = Float64,
    kwargs...,
)
    w = ExaModels.ExaCore(T; backend = backend)
    va = ExaModels.variable(w, length(data.bus))
    vm = ExaModels.variable(
        w,
        length(data.bus);
        start = data.vm0,
        lvar = data.vmin,
        uvar = data.vmax,
    )
    pg = ExaModels.variable(w, length(data.gen); start=data.pg0, lvar = data.pmin, uvar = data.pmax)
    qg = ExaModels.variable(w, length(data.gen); start=data.qg0, lvar = data.qmin, uvar = data.qmax)
    p = ExaModels.variable(w, length(data.arc); lvar = -data.rate_a, uvar = data.rate_a)
    q = ExaModels.variable(w, length(data.arc); lvar = -data.rate_a, uvar = data.rate_a)

    o = ExaModels.objective(
        w,
        g.cost1 * pg[g.i]^2 + g.cost2 * pg[g.i] + g.cost3 for g in data.gen
    )

    c1 = ExaModels.constraint(w, va[i] for i in data.ref_buses)

    # Active power flow, FR
    c2 = ExaModels.constraint(
        w,
        p[b.f_idx] - b.c5 * vm[b.f_bus]^2 -
        b.c3 * (vm[b.f_bus] * vm[b.t_bus] * cos(va[b.f_bus] - va[b.t_bus])) -
        b.c4 * (vm[b.f_bus] * vm[b.t_bus] * sin(va[b.f_bus] - va[b.t_bus])) for
        b in data.branch
    )
    # Reactive power flow, FR
    c3 = ExaModels.constraint(
        w,
        q[b.f_idx] +
        b.c6 * vm[b.f_bus]^2 +
        b.c4 * (vm[b.f_bus] * vm[b.t_bus] * cos(va[b.f_bus] - va[b.t_bus])) -
        b.c3 * (vm[b.f_bus] * vm[b.t_bus] * sin(va[b.f_bus] - va[b.t_bus])) for
        b in data.branch
    )
    # Active power flow, TO
    c4 = ExaModels.constraint(
        w,
        p[b.t_idx] - b.c7 * vm[b.t_bus]^2 -
        b.c1 * (vm[b.t_bus] * vm[b.f_bus] * cos(va[b.t_bus] - va[b.f_bus])) -
        b.c2 * (vm[b.t_bus] * vm[b.f_bus] * sin(va[b.t_bus] - va[b.f_bus])) for
        b in data.branch
    )
    # Reactive power flow, TO
    c5 = ExaModels.constraint(
        w,
        q[b.t_idx] +
        b.c8 * vm[b.t_bus]^2 +
        b.c2 * (vm[b.t_bus] * vm[b.f_bus] * cos(va[b.t_bus] - va[b.f_bus])) -
        b.c1 * (vm[b.t_bus] * vm[b.f_bus] * sin(va[b.t_bus] - va[b.f_bus])) for
        b in data.branch
    )

    # Voltage angle difference
    c6 = ExaModels.constraint(
        w,
        va[b.f_bus] - va[b.t_bus] for b in data.branch;
        lcon = data.angmin,
        ucon = data.angmax,
    )
    # Line flow constraints
    c7 = ExaModels.constraint(
        w,
        p[b.f_idx]^2 + q[b.f_idx]^2 - b.rate_a_sq for b in data.branch;
        lcon = fill!(similar(data.branch, Float64, length(data.branch)), -Inf),
    )
    c8 = ExaModels.constraint(
        w,
        p[b.t_idx]^2 + q[b.t_idx]^2 - b.rate_a_sq for b in data.branch;
        lcon = fill!(similar(data.branch, Float64, length(data.branch)), -Inf),
    )

    # Active power balance
    c9 = ExaModels.constraint(w, b.pd + b.gs * vm[b.i]^2 for b in data.bus)
    c11 = ExaModels.constraint!(w, c9, a.bus => p[a.i] for a in data.arc)
    c13 = ExaModels.constraint!(w, c9, g.bus => -pg[g.i] for g in data.gen)
    # Reactive power balance
    c10 = ExaModels.constraint(w, b.qd - b.bs * vm[b.i]^2 for b in data.bus)
    c12 = ExaModels.constraint!(w, c10, a.bus => q[a.i] for a in data.arc)
    c14 = ExaModels.constraint!(w, c10, g.bus => -qg[g.i] for g in data.gen)

    return ExaModels.ExaModel(w; kwargs...)
end

acopf_model (generic function with 1 method)

Before diving into the solution on the GPU, we show how to solve `case9`
using MadNLP:

In [4]:
using MadNLP

nlp = acopf_model(data)
results = madnlp(nlp)
nothing

This is MadNLP version v0.8.7, running with umfpack

Number of nonzeros in constraint Jacobian............:      295
Number of nonzeros in Lagrangian Hessian.............:      417

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       51
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:       27
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        9
        inequality constraints with only upper bounds:       18

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.4455294e+03 1.63e+00 3.93e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  5.2194864e+03 1.70e+00 3.27e+00  -1.0 1.30e+00    -  6.68e-01 1.00e+00h  

## Solving optimal power flow on the GPU
For solving the optimal power flow model on the GPU, the set-up is similar to
what we have detailed in the tutorial 3. We start by importing MadNLPGPU, and we
instantiate a new optimal power flow instance on the GPU:

In [5]:
using CUDA
using MadNLPGPU

nlp_gpu = acopf_model(data; backend=CUDABackend())

An ExaModel{Float64, CUDA.CuArray{Float64, 1, CUDA.DeviceMemory}, ...}

  Problem name: Generic
   All variables: ████████████████████ 60     All constraints: ████████████████████ 82    
            free: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: █████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 18    
         low/upp: █████████████████⋅⋅⋅ 51             low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ██████████████⋅⋅⋅⋅⋅⋅ 55    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 77.21% sparsity)   417             linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ████████████████████ 82    
                                                         nnzj: ( 94.00% sparsity)   2

Solving the problem using cuDSS simply amounts to

In [6]:
results = madnlp(nlp_gpu)
nothing

This is MadNLP version v0.8.7, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:      295
Number of nonzeros in Lagrangian Hessian.............:      417

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       51
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:       27
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        9
        inequality constraints with only upper bounds:       18

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.4455294e+03 1.63e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  5.4455293e+03 1.63e+00 9.28e-03  -1.0 1.44e+00    -  6.45e-01 2.00e-

The instance `case9` is too small to get any significant speed-up compared
to the CPU. However, we can solve a larger instance just by importing new data:
For instance, to solve the case `10000_goc`:

In [7]:
data = JLD2.load(joinpath(DATA_DIR, "pglib_opf_case10000_goc.jld2"))["data"]
nlp_gpu = acopf_model(data; backend=CUDABackend())
results = madnlp(nlp_gpu)
nothing

This is MadNLP version v0.8.7, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:   419823
Number of nonzeros in Lagrangian Hessian.............:   602508

Total number of variables............................:    76804
                     variables with only lower bounds:        0
                variables with lower and upper bounds:    66804
                     variables with only upper bounds:        0
Total number of equality constraints.................:    72773
Total number of inequality constraints...............:    39579
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:    13193
        inequality constraints with only upper bounds:    26386

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.0503288e+06 8.97e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  3.0503288e+06 8.97e+00 4.39e-02  -1.0 1.99e+01    -  5.52e-02 2.71e-

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*